# Reflectancia TOA ASTER L1T GEE

In [1]:
# Importar paquete GEE
import ee
ee.Initialize()

## Filtro zona estudio mediante GEE

In [2]:
# Agregar un shp GEE
zona_ee = ee.FeatureCollection("users/bravomoralesnino/SHP/ZONA_19S")

In [3]:
# Importar geemap
import geemap
Map = geemap.Map(basemap='ESRI')

In [4]:
geometria = zona_ee.geometry()
Map.centerObject(geometria, 11)

In [5]:
Map.addLayer(zona_ee, {"color" : "00FF11"}, name = "Zona")
Map

Map(center=[-12.908744902436744, -69.9947155021359], controls=(WidgetControl(options=['position', 'transparent…

## Seleccion de imagen ASTER L1T

In [6]:
# Seleccionar una imagen
ASTER_img = ee.Image("ASTER/AST_L1T_003/20070730150418")\
              .select(["B01","B02","B3N","B04","B05","B06","B07","B08","B09"])\
              .clip(zona_ee)

In [7]:
# Nombre de las bandas
print(ASTER_img.bandNames().getInfo())

['B01', 'B02', 'B3N', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09']


In [8]:
# Proyeccion se encuentra zona y resolucion espacial 15m B1
ASTER_img.select(["B01"]).projection().getInfo()

{'type': 'Projection',
 'crs': 'EPSG:32619',
 'transform': [15, 0, 333083, 0, -15, -1388422]}

In [9]:
# Proyeccion se encuentra zona y resolucion espacial 30m B1
ASTER_img.select(["B06"]).projection().getInfo()

{'type': 'Projection',
 'crs': 'EPSG:32619',
 'transform': [30, 0, 333075, 0, -30, -1388415]}

In [10]:
# Reproyeccion segun zona
ASTER_img_utm = ASTER_img.reproject(crs="EPSG:32719", scale = 15)

In [15]:
# Consultar proyeccion
ASTER_img_utm.projection().getInfo()

{'type': 'Projection', 'crs': 'EPSG:32719', 'transform': [15, 0, 0, 0, -15, 0]}

In [16]:
# Simbologia ND
viz = {
    'bands': ['B07', 'B3N', 'B02'],
    'min': 30,
    'max': 150,
    'gamma': 1.8
}

In [17]:
# Visualizar 
Map.addLayer(ASTER_img_utm, viz,name = "ASTER L1T Rad")
Map

Map(bottom=281405.0, center=[-12.908744902436744, -69.9947155021359], controls=(WidgetControl(options=['positi…

## Seleccion de las bandas

In [18]:
# Seleccionar las bandas
B1 = ASTER_img_utm.select("B01")
B2 = ASTER_img_utm.select("B02")
B3 = ASTER_img_utm.select("B3N")
B4 = ASTER_img_utm.select("B04")
B5 = ASTER_img_utm.select("B05")
B6 = ASTER_img_utm.select("B06")
B7 = ASTER_img_utm.select("B07")
B8 = ASTER_img_utm.select("B08")
B9 = ASTER_img_utm.select("B09")

## Convertir a Randiancia Sensor

In [20]:
# Del 2008 adelante no hay bandas 05 al 09
gain_B01 = ASTER_img_utm.get('GAIN_COEFFICIENT_B01').getInfo()
gain_B02 = ASTER_img_utm.get('GAIN_COEFFICIENT_B02').getInfo()
gain_B03 = ASTER_img_utm.get('GAIN_COEFFICIENT_B3N').getInfo()
gain_B04 = ASTER_img_utm.get('GAIN_COEFFICIENT_B04').getInfo()
gain_B05 = ASTER_img_utm.get('GAIN_COEFFICIENT_B05').getInfo()
gain_B06 = ASTER_img_utm.get('GAIN_COEFFICIENT_B06').getInfo()
gain_B07 = ASTER_img_utm.get('GAIN_COEFFICIENT_B07').getInfo()
gain_B08 = ASTER_img_utm.get('GAIN_COEFFICIENT_B08').getInfo()
gain_B09 = ASTER_img_utm.get('GAIN_COEFFICIENT_B09').getInfo()

In [21]:
# Determinar radiancia
L1 = B1.subtract(1).multiply(gain_B01)
L2 = B2.subtract(1).multiply(gain_B02)
L3 = B3.subtract(1).multiply(gain_B03)
L4 = B4.subtract(1).multiply(gain_B04)
L5 = B5.subtract(1).multiply(gain_B05)
L6 = B6.subtract(1).multiply(gain_B06)
L7 = B7.subtract(1).multiply(gain_B07)
L8 = B8.subtract(1).multiply(gain_B08)
L9 = B9.subtract(1).multiply(gain_B09)

## Determinar día juliano ASTER L1T

In [22]:
# Determinar dia juliano ASTER L1T
ID_ASTER = ASTER_img.get("system:id").getInfo()
ID_ASTER

'ASTER/AST_L1T_003/20070730150418'

In [24]:
year = int(ID_ASTER[18:22])
year

2007

In [25]:
month = int(ID_ASTER[22:24])
month

7

In [26]:
day = int(ID_ASTER[24:26])
day

30

In [27]:
import datetime as dt

In [28]:
fecha = dt.datetime(year,month,day)
fecha

datetime.datetime(2007, 7, 30, 0, 0)

In [29]:
dia_juliano = int(fecha.strftime("%j"))
dia_juliano

211

## Determinar distancia sol a tierra "d"

In [30]:
import math
# Determinar la distancia del sol a tierra
d = 1 + 0.0167*(math.sin((2*math.pi*(dia_juliano - 93.5))/365))
d

1.0150238355231924

## Determinar angulo elevación solar

In [31]:
# Determinar elevacion solar
SOLAR_ELEVATION = ASTER_img.get("SOLAR_ELEVATION").getInfo()
SOLAR_ELEVATION

49.541722

In [32]:
# Conversion angulo de elevacion solar a radianes
SOLAR_ELEVATION_R = SOLAR_ELEVATION*math.pi/180
SOLAR_ELEVATION_R

0.8646661660077102

## ESUN bandas ASTER L1T

In [33]:
# ESUN ASTER L1T
ESUN = [1848,1549,1114,225.4,86.63,81.85,74.85,66.49,59.85]

In [36]:
ESUN[0]

1848

## Determinación reflectancia TOA 

In [38]:
d*d

1.0302733866802127

In [39]:
# Banda Reflectancia TOA GEE
pi = ee.Image(math.pi)
SUN_ELEVATION_ee = ee.Number(SOLAR_ELEVATION_R)
d = ee.Image(d*d)
esun1 = ee.Image(ESUN[0])
esun2 = ee.Image(ESUN[1])
esun3 = ee.Image(ESUN[2])
esun4 = ee.Image(ESUN[3])
esun5 = ee.Image(ESUN[4])
esun6 = ee.Image(ESUN[5])
esun7 = ee.Image(ESUN[6])
esun8 = ee.Image(ESUN[7])
esun9 = ee.Image(ESUN[8])

In [40]:
# Determinar reflectancia TOA
TOA_B1 = L1.multiply(pi).multiply(d).divide(esun1.multiply(SUN_ELEVATION_ee.sin()))
TOA_B2 = L2.multiply(pi).multiply(d).divide(esun2.multiply(SUN_ELEVATION_ee.sin()))
TOA_B3 = L3.multiply(pi).multiply(d).divide(esun3.multiply(SUN_ELEVATION_ee.sin()))
TOA_B4 = L4.multiply(pi).multiply(d).divide(esun4.multiply(SUN_ELEVATION_ee.sin()))
TOA_B5 = L5.multiply(pi).multiply(d).divide(esun5.multiply(SUN_ELEVATION_ee.sin()))
TOA_B6 = L6.multiply(pi).multiply(d).divide(esun6.multiply(SUN_ELEVATION_ee.sin()))
TOA_B7 = L7.multiply(pi).multiply(d).divide(esun7.multiply(SUN_ELEVATION_ee.sin()))
TOA_B8 = L8.multiply(pi).multiply(d).divide(esun8.multiply(SUN_ELEVATION_ee.sin()))
TOA_B9 = L9.multiply(pi).multiply(d).divide(esun9.multiply(SUN_ELEVATION_ee.sin()))

## Composicion de bandas

In [41]:
# Compilacion de las bandas
TOA_B123456789 = TOA_B1.addBands([TOA_B2,TOA_B3,TOA_B4,TOA_B5,
                                  TOA_B6,TOA_B7,TOA_B8,TOA_B9])

In [42]:
# Simbologia TOA
viz_toa = {
    'bands': ['B07', 'B3N', 'B02'],
    'min': 0.1,
    'max': 0.6,
    'gamma': 1.8
}

In [43]:
# Visualizar 
Map.addLayer(TOA_B123456789, viz_toa,name = "ASTER L1T TOA")
Map

Map(bottom=281402.2727050781, center=[-12.906859517826103, -69.90222930908205], controls=(WidgetControl(option…

## Exportar Imagen calibrado

In [44]:
import os

In [45]:
os.getcwd()

'C:\\Users\\GEOMATICA\\Teledeteccion_Python\\GEE'

In [46]:
ruta_archivos = r"E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE"
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\Resultado_GEE'

In [47]:
# Exportar en google drive
geemap.ee_export_image_to_drive(TOA_B123456789, description='ASTER_TOA_B123456789_clip', folder='GEE_Python', region=geometria, scale=15)

Exporting ASTER_TOA_B123456789_clip ...
